# Import Libraries

In [126]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.image import imread
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, MobileNetV2
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import os
import tensorflow.keras as keras

# Data Loading and Preprocessing

In [127]:
# Directories for training, testing, validation and prediction datasets
train_dir = '/Users/prithvishams/Documents/Bootcamp Concordia/Project Bird Classification/Bird Data/train'
test_dir = '/Users/prithvishams/Documents/Bootcamp Concordia/Project Bird Classification/Bird Data/test'
val_dir = '/Users/prithvishams/Documents/Bootcamp Concordia/Project Bird Classification/Bird Data/valid'
predict_dir = '/Users/prithvishams/Documents/Bootcamp Concordia/Project Bird Classification/Bird Data/predict'

In [128]:
# Model parameters
img_size = (224, 224)
batch_size = 32
num_classes = len([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])

num_classes

20

In [129]:
# Data augmentation for training
train_img_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# No augmentation for val/test data
val_test_img_gen = ImageDataGenerator(rescale=1./255)

In [130]:
# Load training data
train_data = train_img_gen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)
# Load validation data
val_data = val_test_img_gen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)
# Load test data
test_data = val_test_img_gen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 3208 images belonging to 20 classes.
Found 100 images belonging to 20 classes.
Found 100 images belonging to 20 classes.


# Model Construction

In [131]:
# Load MobileNetV2 model with pre-trained weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
# Freeze the base model
base_model.trainable = False

In [132]:
# Add custom layers on top of the base model
input_layer = layers.Input(shape=(224, 224, 3)) #input layer
x = base_model(input_layer, training = False) #base model runs in inference mode
x = layers.GlobalAveragePooling2D()(x) # flattens 7x7x1280 to 1280 by taking average of each 7x7 block
x = layers.Dropout(0.2)(x) # dropout layer to prevent overfitting
output_layer = layers.Dense(num_classes, activation='softmax')(x) # output layer with softmax activation

# Create the model
model = models.Model(inputs=input_layer, outputs=output_layer)



# Compile and Train Model

In [133]:
# Compile model

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [134]:
# Train Model

early_stopping = EarlyStopping(monitor='val_loss', patience=4)
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=[early_stopping]
)


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 23s 211ms/step - accuracy: 0.4220 - loss: 2.0941 - val_accuracy: 0.9000 - val_loss: 0.3788
Epoch 2/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 22s 213ms/step - accuracy: 0.8836 - loss: 0.4494 - val_accuracy: 0.9300 - val_loss: 0.2436
Epoch 3/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 22s 221ms/step - accuracy: 0.9153 - loss: 0.3085 - val_accuracy: 0.9400 - val_loss: 0.1779
Epoch 4/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 23s 229ms/step - accuracy: 0.9252 - loss: 0.2602 - val_accuracy: 0.9300 - val_loss: 0.1649
Epoch 5/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 24s 236ms/step - accuracy: 0.9331 - loss: 0.2202 - val_accuracy: 0.9500 - val_loss: 0.1603
Epoch 6/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 24s 239ms/step - accuracy: 0.9413 - loss: 0.2043 - val_accuracy: 0.9600 - val_loss: 0.1429
Epoch 7/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 25s 241ms/step - accuracy: 0.9510 - loss: 0.1594 - val_accuracy: 0.9600 - val_loss: 0.1369
Epoch 8/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 25s 244ms/step - accuracy: 0.9668 - loss: 0

In [135]:
# Evaluate on Test Data
test_loss, test_accuracy = model.evaluate(test_data)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy*100:.2f}%')

# Save the model
model.save('bird_classification_model.h5')

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.9753 - loss: 0.0613


Test Loss: 0.050866927951574326
Test Accuracy: 98.00%


# Predict New Images

In [136]:
# Load Prediction Images
predict_data = keras.utils.image_dataset_from_directory(
    predict_dir,
    labels=None,
    image_size=(224, 224),
    batch_size=1,
    shuffle=False
) # no subdirectory required when labels = None

# Make predictions
predictions = model.predict(predict_data)

Found 6 files.
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [22]:
predictions.shape

(6, 20)

In [137]:
# Get predicted class indices
predicted_class_indices = np.argmax(predictions, axis=1)
predicted_class_indices

array([11, 11, 11, 11, 11, 17])

In [77]:
# Map class indices to class labels
class_names = list(train_data.class_indices.keys())
class_names

['ABBOTTS BABBLER',
 'ABBOTTS BOOBY',
 'ABYSSINIAN GROUND HORNBILL',
 'AFRICAN CROWNED CRANE',
 'AFRICAN EMERALD CUCKOO',
 'AFRICAN FIREFINCH',
 'AFRICAN OYSTER CATCHER',
 'AFRICAN PIED HORNBILL',
 'AFRICAN PYGMY GOOSE',
 'ALBATROSS',
 'ALBERTS TOWHEE',
 'ALEXANDRINE PARAKEET',
 'ALPINE CHOUGH',
 'ALTAMIRA YELLOWTHROAT',
 'AMERICAN AVOCET',
 'AMERICAN BITTERN',
 'AMERICAN COOT',
 'AMERICAN FLAMINGO',
 'AMERICAN GOLDFINCH',
 'AMERICAN KESTREL']

In [138]:
predicted_labels = [class_names[i] for i in predicted_class_indices]
predicted_labels
# Create a DataFrame for predictions
predictions_df = pd.DataFrame({
    'Image': [os.path.basename(f) for f in predict_data.file_paths],
    'Predicted Label': predicted_labels
})
predictions_df

,Image,Predicted Label
0,1.jpg,ALEXANDRINE PARAKEET
1,2.jpg,ALEXANDRINE PARAKEET
2,3.jpg,ALEXANDRINE PARAKEET
3,4.jpg,ALEXANDRINE PARAKEET
4,5.jpg,ALEXANDRINE PARAKEET
5,6.jpg,AMERICAN FLAMINGO
